## 【GPT4o】微调实战初窥之写作风格

可能大家听过一些微调的专业方法：

1. 全量微调 (Full Fine-tuning)
- 对预训练模型的所有参数进行更新和调整，就像重新装修整栋房子，从地板到天花板都可以改造


2. 参数高效微调 (PEFT - Parameter Efficient Fine-tuning)
- LoRA (Low-Rank Adaptation)
  * 通过低秩分解来优化关键参数矩阵，不是改造整栋房子，而是只在关键位置安装智能开关和控制器


- Prefix Tuning/Prompt Tuning
  * 在输入端增加可训练的连续提示向量，像是给房子装了一个智能管家，通过不同的指令来调控房子的各个功能


- Adapter Tuning
  * 在原有网络层之间插入小型可训练模块，在房子的各个房间之间安装智能中控器，不改变房间本身结构


3. 指令微调 (Instruction Tuning)
- 通过指令数据集来增强模型对自然语言指令的理解，教会智能管家理解更多样化的人类指令，像是"把客厅温度调高一点"而不是必须说"温度调到26度"


4. 增量预训练 (Incremental Pre-training)
- 专业解释：在特定领域数据上继续预训练，给房子增加新功能区域，比如把普通客厅改造成家庭影院

这些方法的选择主要取决于：
1. 计算资源限制
2. 任务特殊性要求
3. 数据规模大小
4. 是否需要多任务共享

比如:
- 资源充足追求效果→全量微调
- 资源受限重视效率→LoRA
- 多任务快速切换→Prompt Tuning
- 需要模块化组合→Adapter
- 提升通用理解→指令微调
- 领域深度适应→增量预训练

上面这些主要适用于开源模型，是更底层更定制化的微调方法,是在我们课程体系中，但我们现在并不涉及这些底层微调的讲解，这一课程板块主要讲在线模型的微调使用。

但与上面的具体方法纬度不同，OpenAI 的 API 创建的是一个监督式微调（Supervised Fine-tuning）任务，这是什么呢，和上面我们常听到的有什么不同？

1. 基础预训练 (Pre-training)
- 在大规模无标注数据上进行自监督学习，像是让一个人通过大量阅读来建立基础知识体系
- 特点：
  * 无需人工标注，学习通用知识表示，计算成本最高，需要海量数据

2. 有监督微调 (Supervised Fine-tuning)
- 使用人工标注的输入-输出对来定向训练模型，像是请专业老师有针对性地教学生解题
- 实现方式可以是：
  * 全量微调：调整所有参数
  * LoRA：只训练关键低秩矩阵
  * Adapter：插入小型可训练模块
- 特点：
  * 需要高质量标注数据，训练目标明确，效果可控可预期

3. 强化学习 (RL)
- 使用奖惩反馈来优化模型输出，像是给学生即时反馈，表扬好的回答，纠正不当表述。目前有在实时推理（o1思考链）中应用的趋势



形象表示：

预训练 → SFT

先让学生广泛阅读，再进行针对性培训。适用于任务明确、有标准答案的场景


**监督式微调**

```python
client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model=MODEL,
)
```


- **利用标注数据进行训练**：在这个微调过程中，提供了一个 `training_file`，其中包含了输入和期望输出的配对数据。这意味着模型将根据这些已知的问题-回答对来调整自身参数，以更好地模拟这种映射关系。
  
- **调整模型参数以适应特定任务**：监督式微调的核心是通过已标注的数据来指导模型学习特定的任务需求，例如特定的回答风格、领域知识或任务指令。

- **使用预训练模型作为基础**：指定了一个预训练的 `MODEL`。微调过程是在这个预训练模型的基础上进一步训练，使其更贴合的特定应用场景。

### **详细解释**

- **微调的目的**：通过微调，可以让模型在特定领域或任务上表现得更好。例如，让模型更好地理解医疗领域的术语，或者按照特定的格式生成响应。

- **监督式的特征**：监督式微调需要提供一组已知的输入和期望的输出。这些数据指导模型学习正确的响应模式。

- **OpenAI 的微调机制**：在 OpenAI 的微调 API 中，上传的 `training_file` 应该符合特定的格式，通常是一个 JSON Lines 文件，其中每一行包含一个完整的message。


流程步骤

收集原始数据：首先，收集与目标任务相关的原始数据。这可以是对话数据、分类数据、生成任务数据等，具体取决于的任务类型。确保数据集具有代表性和多样性，以提高模型的泛化能力。

划分数据集（可选）：将标注数据划分为训练集、验证集和测试集。通常，大部分数据用于训练，一小部分用于验证模型的性能和调整超参数，最后一部分用于最终评估模型的泛化能力。

数据预处理：根据任务的要求，对数据进行预处理。这可能包括文本清洗、分词、去除停用词、词干化等处理步骤。确保数据格式和特征表示适合模型的输入要求。

格式转换：将数据转换为适合模型训练的格式。这可能涉及将数据转换为文本文件、JSON格式或其他适合模型输入的格式。

模型微调：上传微调语料，开启微调job。

模型评估与应用：如我们调整写文章风格的将微调后的模型应用于实际任务用看是最合适的，其他的可以创建正确答案比对等合适的评判标准。

官方文档：https://platform.openai.com/docs/guides/fine-tuning

中文语料集：https://huggingface.co/datasets/liwu/MNBVC

有些文档，只有“专业的人”才能写，至少目前的大模型是不行的，比如产品经理的周报，程序员的周报，组织的报告。。。

In [7]:
import openai
import os

client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))


In [27]:
from datasets import load_dataset
from itertools import islice

# 流式加载数据集
dataset = load_dataset("liwu/MNBVC", 'law_judgement', split='train', streaming=True, trust_remote_code=True)

# 使用 islice 获取前10条记录
data = list(islice(dataset, 10))

# 打印前10条数据
print(data)

[{'text': '黑龙江省齐齐哈尔市梅里斯达斡尔区人民法院\n\n民 事 判 决 书\n\n（2016）黑0208民初1497号\n\n原告王某某，男，1996年6月5日出生，汉族，农民。\n\n委托代理人王某某，系黑龙江发达律师事务所律师。\n\n被告中国人民财产保险股份有限公司齐齐哈尔市梅里斯达斡尔族区支公司，住所地黑龙江省齐齐哈尔市梅里斯达斡尔族区城镇。\n\n代表人王某某，系该公司经理。\n\n委托代理人李某某，系黑龙江宇恒律师事务所律师。\n\n委托代理人张某某，系黑龙江宇恒律师事务所律师。\n\n被告吕某，男，1987年2月6日出生，汉族，农民。\n\n原告王某某与被告中国人民财产保险股份有限公司齐齐哈尔市梅里斯达斡尔族区支公司[以下简称人保财险梅里斯支公司]、吕某机动车交通事故责任纠纷一案,原告于2016年9月14日向本院提起诉讼，本院受理后，依法适用普通程序由审判员马国富担任审判长，与人民陪审员毕从志、吴淑霞共同组成合议庭，于2016年10月19日公开开庭进行了审理。原告王某某委托代理人王某某、被告人保财险梅里斯支公司委托代理人张某某到庭参加了诉讼。被告吕某经本院传票传唤无正当理由未到庭参加诉讼。本案现已缺席审理终结。\n\n原告王某某诉称，2014年5月24日16时许，被告吕某驾驶黑BV86**号轿车，在梅里斯区育德街与碾北公路交叉口由北向东左转弯时，与沿碾北公路由西向东原告驾驶的无号牌两轮摩托车相撞，造成双方车辆损坏，原告受伤的后果。原告在齐齐哈尔市公安医院住院治疗7天，出院诊断为：1、急性闭合性颅脑损伤；2、左颞顶部硬膜外血肿；3、左颞顶固骨折；4、左颞头皮下血肿伴擦皮伤；5、右肘擦皮伤；住院期间，一级护理4天、二级护理3天。经齐齐哈尔市梅里斯区交通警察大队《道路交通事故认定书》（简易程序）第2302083201400018号认定：被告吕某对该起事故负主要责任。被告吕某驾驶的肇事车辆在人保财险梅里斯支公司处投保了交强险，且事故发生在保险期间内。现要求被告人保财险梅里斯支公司在机动车强制保险责任范围内赔偿原告医药费8993.00元、护理费1573.00元、交通费200.00元、误工费1001.00元、伙食补助费700.00元，合计12467.00元，并要求被告吕某承担连带赔偿责任。\n\n原告为证明其诉讼主张，提供了如下证据：\n\n1、交通

In [65]:
import json
formatted_data = []
text = ""
for entry in data:
    try:
        text = entry['text']
        try:
            first_newline_index = text.index('\n\n')
            start_index = first_newline_index + 2
            second_newline_index = text.index('\n\n', start_index)
            end_index = second_newline_index
            file_name=text[start_index:end_index].strip()
        except ValueError:
            formatted_data.append(None)
    except KeyError:
        formatted_data.append(None)


    # 去除最后可能多余的换行符
    text = text.strip()

    # 格式化为问答对
    question = {"role": "user", "content": f"写一篇{file_name}"}
    answer = {"role": "assistant", "content": text}

    # 添加系统提示
    system = {"role": "system", "content": "按法律判决书的口吻写"}

    formatted_data.append({"messages": [system, question, answer]})


KeyboardInterrupt: 

In [32]:
formatted_data

[{'messages': [{'role': 'system', 'content': '按法律判决书的口吻写'},
   {'role': 'user', 'content': '写一篇民 事 判 决 书'},
   {'role': 'assistant',
    'content': '黑龙江省齐齐哈尔市梅里斯达斡尔区人民法院\n\n民 事 判 决 书\n\n（2016）黑0208民初1497号\n\n原告王某某，男，1996年6月5日出生，汉族，农民。\n\n委托代理人王某某，系黑龙江发达律师事务所律师。\n\n被告中国人民财产保险股份有限公司齐齐哈尔市梅里斯达斡尔族区支公司，住所地黑龙江省齐齐哈尔市梅里斯达斡尔族区城镇。\n\n代表人王某某，系该公司经理。\n\n委托代理人李某某，系黑龙江宇恒律师事务所律师。\n\n委托代理人张某某，系黑龙江宇恒律师事务所律师。\n\n被告吕某，男，1987年2月6日出生，汉族，农民。\n\n原告王某某与被告中国人民财产保险股份有限公司齐齐哈尔市梅里斯达斡尔族区支公司[以下简称人保财险梅里斯支公司]、吕某机动车交通事故责任纠纷一案,原告于2016年9月14日向本院提起诉讼，本院受理后，依法适用普通程序由审判员马国富担任审判长，与人民陪审员毕从志、吴淑霞共同组成合议庭，于2016年10月19日公开开庭进行了审理。原告王某某委托代理人王某某、被告人保财险梅里斯支公司委托代理人张某某到庭参加了诉讼。被告吕某经本院传票传唤无正当理由未到庭参加诉讼。本案现已缺席审理终结。\n\n原告王某某诉称，2014年5月24日16时许，被告吕某驾驶黑BV86**号轿车，在梅里斯区育德街与碾北公路交叉口由北向东左转弯时，与沿碾北公路由西向东原告驾驶的无号牌两轮摩托车相撞，造成双方车辆损坏，原告受伤的后果。原告在齐齐哈尔市公安医院住院治疗7天，出院诊断为：1、急性闭合性颅脑损伤；2、左颞顶部硬膜外血肿；3、左颞顶固骨折；4、左颞头皮下血肿伴擦皮伤；5、右肘擦皮伤；住院期间，一级护理4天、二级护理3天。经齐齐哈尔市梅里斯区交通警察大队《道路交通事故认定书》（简易程序）第2302083201400018号认定：被告吕某对该起事故负主要责任。被告吕某驾驶的肇事车辆在人保财险梅里斯支公司处投保了交强险，且事故发生在保险期间内。现要求被告人保财险梅里斯支

将数据保存为 `.jsonl` 文件，每行是一个训练示例对话。

In [33]:
def save_to_jsonl(data, filename):
    """将变量中的数据保存到 JSON Lines (JSONL) 格式的文件中。

    Args:
        data: 一个可迭代对象，其中每个元素都是一个字典或可转换为字典的对象。
        filename: 要保存的文件名（包含 .jsonl 扩展名）。
    """

    with open(filename, 'w', encoding='utf-8') as f:
        for item in data:
            try:
                json_str = json.dumps(item, ensure_ascii=False)  # ensure_ascii=False 支持中文等非ASCII字符
                f.write(json_str + '\n')
            except TypeError as e:
                print(f"无法将以下数据转换为JSON: {item}\n错误: {e}")

In [34]:
save_to_jsonl(formatted_data, "formatted_data.jsonl")

---

## 微调

在确保拥有正确数量和结构的数据集，并已上传文件后，下一步是创建一个微调作业。我们支持通过微调 UI或以编程方式创建微调作业。

现在，我们可以使用生成的文件和一个可选的后缀来创建微调作业，以便识别模型。响应将包含一个 `id`，可以使用它来检索作业的更新。

**注意**：文件需要先被我们的系统处理，可能会收到“File not ready”的错误。在这种情况下，请稍等几分钟后重试。

https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken

In [35]:
import json
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

def count_content_tokens_in_jsonl(filename):
    """计算 JSONL 文件中 'content' 字段的 token 数量。"""
    total_tokens = 0
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data = json.loads(line)
                if "messages" in data and isinstance(data["messages"], list):  # 检查 "messages" 键是否存在且为列表
                    for message in data["messages"]:
                        if "content" in message and isinstance(message["content"], str): # 检查"content"键和字符串类型
                            tokens = encoding.encode(message["content"])
                            total_tokens += len(tokens)
            except json.JSONDecodeError as e:
                print(f"JSON解码错误: {e}，跳过此行: {line.strip()}")
    return total_tokens



In [36]:
filename = "formatted_data.jsonl" 
token_count = count_content_tokens_in_jsonl(filename)
print(f"文件 '{filename}' 中 'content' 字段的 token 数量: {token_count}")

文件 'formatted_data.jsonl' 中 'content' 字段的 token 数量: 18585


gpt-4o-mini是3.000 美元/100 万训练token，https://openai.com/api/pricing/

In [37]:
with open("formatted_data.jsonl", 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

我们可以执行一系列错误检查，以验证数据集中的每个对话是否符合微调 API 所需的格式。错误将根据其性质进行分类，方便调试。

1. **数据类型检查**：检查数据集中的每个条目是否是字典 (`dict`)。错误类型：`data_type`。
2. **消息列表检查**：检查每个条目中是否存在 `messages` 列表。错误类型：`missing_messages_list`。
3. **消息键检查**：验证 `messages` 列表中的每条消息是否包含 `role` 和 `content` 键。错误类型：`message_missing_key`。
4. **消息中未识别的键**：记录消息中是否存在 `role`、`content`、`weight`、`function_call` 和 `name` 之外的键。错误类型：`message_unrecognized_key`。
5. **角色验证**：确保 `role` 只包含 "system"、"user" 或 "assistant"。错误类型：`unrecognized_role`。
6. **内容验证**：验证 `content` 是否为字符串类型且包含文本数据。错误类型：`missing_content`。
7. **助手消息检查**：确保每个对话至少有一条来自助手的消息。错误类型：`example_missing_assistant_message`。

以下代码会执行这些检查，并输出每种类型的错误计数，方便调试并确保数据集准备好进行后续操作。

In [38]:

from collections import defaultdict
format_errors = defaultdict(int)

for ex in formatted_data:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


我们既对花费有了了解，数据格式也校验过了。
接下来就可以正式开始微调啦：

1. 首先通过文件 API上传文件，以便用于微调作业；
2. 接着启动微调的job

In [39]:
training_file_name = "formatted_data.jsonl"
def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id

# 上传训练文件
training_file_id = upload_file(training_file_name, "fine-tune")

print("训练文件 ID:", training_file_id)


训练文件 ID: file-WgdCSBrbee1CWMbkKNYupWAt


启动微调作业

In [40]:
MODEL = "gpt-4o-mini-2024-07-18"

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model=MODEL,
    suffix="recipe-ner",
)

job_id = response.id

print("作业 ID:", response.id)
print("状态:", response.status)

作业 ID: ftjob-kWTFiyPkfNPqqcYh6mreOYTY
状态: validating_files


微调API的更广泛的用法示例，包括设置超参数、实时监控事件以及管理微调模型。  (需要替换示例中的占位符为你自己的文件 ID、作业 ID 和模型名称。) 

In [42]:
# # 列出最多10个微调作业
# client.fine_tuning.jobs.list(limit=10)
# 
# # 获取微调作业的状态
# client.fine_tuning.jobs.retrieve("ftjob-abc123")  # 将 "ftjob-abc123" 替换为你的微调作业 ID
# 
# # 列出微调作业的最多10个事件
# client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-abc123", limit=10) # 将 "ftjob-abc123" 替换为你的微调作业 ID
# 
# # 取消微调作业
# client.fine_tuning.jobs.cancel("ftjob-abc123") # 将 "ftjob-abc123" 替换为你的微调作业 ID
# 
# 
# # 删除微调模型 (必须是创建模型的组织的所有者)
# client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123") # 将模型名称替换为你想要删除的模型


# 使用自定义的超参数创建微调作业
# hyperparams = {
#     "n_epochs": 3,  # 训练轮数
#     "learning_rate_multiplier": 0.1 # 学习率乘数
# }
# fine_tuning_job = client.fine_tuning.jobs.create(training_file="file-XGinujblHPwGLSztz8cPS8XY", model="gpt-3.5-turbo", hyperparameters=hyperparams)
# print(fine_tuning_job)


# 获取特定微调模型的信息
# model = client.models.retrieve("ft:gpt-3.5-turbo-0613:your-org:custom_suffix") # 将模型名称替换为你想要获取信息的模型
# print(model)


In [49]:
client.fine_tuning.jobs.cancel("ftjob-kWTFiyPkfNPqqcYh6mreOYTY")

FineTuningJob(id='ftjob-kWTFiyPkfNPqqcYh6mreOYTY', created_at=1730970649, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-BbylnC34ErFQAfVanaQGLcaB', result_files=[], seed=251141544, status='cancelled', trained_tokens=None, training_file='file-WgdCSBrbee1CWMbkKNYupWAt', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='recipe-ner')

In [67]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-kc7IUjd0IsAFUAzVDnLhnUxU', created_at=1730863716, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-BbylnC34ErFQAfVanaQGLcaB', result_files=[], seed=774007061, status='running', trained_tokens=None, training_file='file-lHL2f8P0QT90EljK0xhwr4qJ', validation_file=None, estimated_finish=1730864074, integrations=[], user_provided_suffix='gov_file'), FineTuningJob(id='ftjob-71V086nBtzUgAo3PxCObxqhz', created_at=1730863609, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-BbylnC34ErFQAfVanaQGLcaB', result_files=[], seed=1

In [43]:
MODEL = "gpt-4o-mini-2024-07-18"

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model=MODEL,
    suffix="gov_file",
)

job_id = response.id

print("作业 ID:", response.id)
print("状态:", response.status)

作业 ID: ftjob-ZSvUCHUf59I8X2UMW3m137Zs
状态: validating_files


#### 检查作业状态

可以对 `https://api.openai.com/v1/alpha/fine-tunes` 端点发出 `GET` 请求，列出的微调作业。在此实例中，需要检查从上一步获得的 ID 是否最终变为 `status: succeeded`。

一旦完成，可以使用 `result_files` 从验证集（如果上传了验证集）中获取结果，并使用 `fine_tuned_model` 参数中的 ID 来调用的微调模型。

In [44]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("作业 ID:", response.id)
print("状态:", response.status)
print("已训练的标记数:", response.trained_tokens)

作业 ID: ftjob-ZSvUCHUf59I8X2UMW3m137Zs
状态: running
已训练的标记数: None


In [66]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("作业 ID:", response.id)
print("状态:", response.status)
print("已训练的标记数:", response.trained_tokens)

作业 ID: ftjob-ZSvUCHUf59I8X2UMW3m137Zs
状态: succeeded
已训练的标记数: 187150


可以通过 `events` 端点跟踪微调的进度。可以多次重新运行下面的代码单元，直到微调完成。

In [52]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 7/100: training loss=0.97
Step 8/100: training loss=1.32
Step 9/100: training loss=1.51
Step 10/100: training loss=1.19
Step 11/100: training loss=1.24
Step 12/100: training loss=1.32
Step 13/100: training loss=1.21
Step 14/100: training loss=0.92
Step 15/100: training loss=1.31
Step 16/100: training loss=1.28
Step 17/100: training loss=0.94
Step 18/100: training loss=0.91
Step 19/100: training loss=1.01
Step 20/100: training loss=0.69
Step 21/100: training loss=1.15
Step 22/100: training loss=0.82
Step 23/100: training loss=0.76
Step 24/100: training loss=0.78
Step 25/100: training loss=0.53
Step 26/100: training loss=0.60


In [55]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 84/100: training loss=0.08
Step 85/100: training loss=0.15
Step 86/100: training loss=0.32
Step 87/100: training loss=0.51
Step 88/100: training loss=0.12
Step 89/100: training loss=0.08
Step 90/100: training loss=0.17
Step 91/100: training loss=0.09
Step 92/100: training loss=0.53
Step 93/100: training loss=0.43
Step 94/100: training loss=0.29
Step 95/100: training loss=0.06
Step 96/100: training loss=0.16
Step 97/100: training loss=0.06
Step 98/100: training loss=0.09
Step 99/100: training loss=0.10
Step 100/100: training loss=0.46
Checkpoint created at step 80
Checkpoint created at step 90
New fine-tuned model created


现在微调完成，我们可以从作业中获取微调模型的 ID：

In [56]:
response = client.fine_tuning.jobs.retrieve(job_id)
response

FineTuningJob(id='ftjob-ZSvUCHUf59I8X2UMW3m137Zs', created_at=1730970697, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=1730971097, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-BbylnC34ErFQAfVanaQGLcaB', result_files=['file-EgfXbcbRZ3B4MRuOFpRu9Ige'], seed=2122530037, status='running', trained_tokens=None, training_file='file-WgdCSBrbee1CWMbkKNYupWAt', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='gov_file')

In [57]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError("未找到微调模型 ID。的作业可能尚未完成。")

print("微调模型 ID:", fine_tuned_model_id)

微调模型 ID: ft:gpt-4o-mini-2024-07-18:acmr:gov-file:AQswdshE


In [58]:
from openai import OpenAI
# 构建 OpenAI 客户端对象的实例
client = OpenAI()

models = client.models.list()
models

SyncPage[Model](data=[Model(id='gpt-4o-mini', created=1721172741, object='model', owned_by='system'), Model(id='gpt-4o-mini-2024-07-18', created=1721172717, object='model', owned_by='system'), Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'), Model(id='o1-preview', created=1725648897, object='model', owned_by='system'), Model(id='gpt-4o-2024-08-06', created=1722814719, object='model', owned_by='system'), Model(id='o1-preview-2024-09-12', created=1725648865, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='gpt-4o', created=1715367049, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='gpt-3.5-turbo', created=1677610602, obj

In [59]:
for model in models:
    print(model.id)

gpt-4o-mini
gpt-4o-mini-2024-07-18
gpt-4-turbo-2024-04-09
o1-preview
gpt-4o-2024-08-06
o1-preview-2024-09-12
dall-e-2
whisper-1
gpt-4o
gpt-3.5-turbo-instruct
tts-1-hd-1106
gpt-3.5-turbo
gpt-3.5-turbo-0125
babbage-002
gpt-4o-2024-05-13
davinci-002
dall-e-3
gpt-4o-realtime-preview
gpt-4o-realtime-preview-2024-10-01
tts-1
tts-1-1106
gpt-3.5-turbo-0301
tts-1-hd
chatgpt-4o-latest
gpt-4-turbo-preview
gpt-4-1106-preview
text-embedding-ada-002
gpt-3.5-turbo-16k
gpt-4-turbo
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-1106
gpt-4o-audio-preview
gpt-3.5-turbo-0613
gpt-4o-audio-preview-2024-10-01
gpt-4-0613
gpt-4-0125-preview
gpt-4
gpt-3.5-turbo-instruct-0914
o1-mini
o1-mini-2024-09-12
ft:gpt-3.5-turbo-0613:acmr:recipe-ner:7rOJnQow
ft:babbage-002:acmr::8LjwL7pi
ft:gpt-3.5-turbo-1106:acmr::8NFcWDKd
ft:gpt-3.5-turbo-1106:acmr::8NG1DFJy
ft:gpt-3.5-turbo-1106:acmr::8NGu7dB7
ft:gpt-3.5-turbo-1106:acmr::8NGecRZn
ft:gpt-3.5-turbo-1106:acmr::8NGoWxGp
ft:gpt-3.5-turbo-

---

## 推理

最后一步是使用的微调模型进行推理。与经典的 `FineTuning` 类似，只需在 `model` 参数中填写新的微调模型名称，调用 `ChatCompletions` 即可。

调用微调模型进行推理：

In [60]:
test_message=[{"role": "user", "content": "以法律判决书的风格写张三醉酒时把李四的车装坏了的判决，要具体真实细节"}]
response = client.chat.completions.create(
    model="gpt-4o-mini", messages=test_message,  max_tokens=800
)
print(response.choices[0].message.content)

**中华人民共和国某某市人民法院**  
**民事判决书**  
**（2023）某民初字第123号**  

**原告：李四，男，XX岁，某某市某某区住址。**  
**被告：张三，男，XX岁，某某市某某区住址。**  

**案件性质：财产损害赔偿纠纷**  

**审理经过**  
原告李四因被告张三醉酒后将其车辆损坏而向本院提起诉讼，要求被告赔偿车辆修理费用及相关损失。案件于2023年10月1日立案，本院依法组成合议庭，于2023年10月15日公开开庭审理本案，原告李四及被告张三均到庭参加诉讼，现已审理终结。

**一、事实及理由**  
原告李四诉称，2023年9月30日晚，其与朋友聚会饮酒，聚会结束后其驾驶的车辆停放在某某市某某区XXX小区门口。2023年10月1日凌晨2时许，被告张三因醉酒驾驶机动车路过该小区，期间与原告李四的车辆发生碰撞，导致原告车辆右侧车门、后视镜及后保险杠等部位受损。原告李四主张，被告应承担全部赔偿责任。

原告提供了以下证据：
1. 事发后报警记录，显示被告张三在车辆碰撞现场；
2. 车辆损坏照片，清晰显示受损情况；
3. 车辆修理公司出具的修理费用明细，共计人民币5000元；
4. 证人证言，出席聚会的王五证实当时原告未饮酒，且未对一旁的车辆进行妨害。

被告张三辩称，事发时并未故意碰撞原告的车辆，认为原告应对车辆停放的位置承担部分责任。对此，原告则表示其车辆正当停放，并未妨碍交通。

**二、法院意见**  
本院认为，依据《中华人民共和国侵权责任法》相关规定，侵权行为导致他人财产损害的，应当承担赔偿责任。被告张三因醉酒驾驶而导致原告李四的车辆受损，属于违法行为，显然应承担损害赔偿责任。

关于被告提出的原告车辆停放位置的问题，经查证，原告车辆停放在合法区域内，不构成对交通的妨碍。被告醉酒驾驶本身已违反交通法规，其应当对此次事故负责。

因此，被告张三应赔偿原告李四车辆修理费用人民币5000元及相关费用。

**三、判决结果**  
根据《中华人民共和国民法典》第1165条、第1166条的规定，判决如下：  
被告张三于本判决生效之日起十日内向原告李四赔偿损失人民币5000元。

如不履行本判决，原告可以依法申请强制执行。

案件受理费由被告张三承担。

本判决为终审判决。

**审判长：XXX**  
**审判员：XXX*

In [61]:
test_message=[{"role": "user", "content": "以法律判决书的风格写张三醉酒时把李四的车装坏了的判决，要具体真实细节"}]
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_message,  max_tokens=1000
)
print(response.choices[0].message.content)

四川省成都市金牛区人民法院

审判员

案件当事人

原告人：张有为，住所地成都市金牛区。

委托代理人：何楠艳，成都泰亘律师事务所律师。

被告人：李四，身份证号码XXXXXXXXXXX，住�

...

成都市金牛区。

委托代理人：唐雅，成都泰亘律师事务所律师。

被告人：杨鑫，身份证号码XXXXXXXXXXX，住址四川省成都市金牛区。

本院在审理原告张有为诉被告李四、杨鑫机动车辆损害责任纠纷一案中，原告张有为被代理人何楠艳、被告李四被代

...

表人唐雅到庭参加诉讼。经合法通知，被告杨鑫未出庭参加诉讼。本案现已审理终结。

原告张有为诉称，2015年9月30日，原告的弟弟张小军因工作需要向被告杨鑫租用其机动车（车牌号：川***YX，型号：�

...

***），租用期为一个月。2015年10月5日，杨鑫驾驶该车前往酒吧陪

...

伴，期间该车与被告李四驾驶的机动车（车牌号：川***H，型号：***，年检类型：非营运，保险公司：中华联合财险四川分�

...

部成都金牛区市场检查点投保三级责任险，保险单号：***，投保起期2015年6月1日，至2016年6月1日）发生碰撞，造成李四�

...

车车辆左后部严重损坏。事发时，杨鑫处于饮酒状态，对事故的发生负有主要责任，事故发生后，双方在事故责任及赔偿事宜上�

...

未协商一致。由于李四要求赔偿的车辆维修费用超过被告杨鑫在上述保险公司投保的相应保险责任限额，导致原告无法从保险�

...

公司获得赔偿。现原告以被告杨鑫为诉讼主体起诉，请求判令被告杨鑫对李四所驾驶车辆的损害赔偿原告金额11,532.73元，

...

该款自判决生效之日起按中国人民银行同期同类贷款基准利率计算罚息至实偿之日。

被告李四辩称，被告杨鑫驾驶其机动车事发时已醉酒，构成故意或明显过失，李四要求赔偿的维修费用均在保险公司的责任�

...

范围内，且已有保险公司定损，李四不同意赔偿原告损失。

被告杨鑫未作答辩。

本院经审理查明，原告张有为的弟弟张小军与被告杨鑫于2015年9月30日签订了机动车租赁合同一份，约定杨鑫将�

...

1台轿车租赁给张小军使用，租赁期自2015年9月30日至2015年10月30日。杨鑫驾驶该车前往酒吧陪伴期间，2015年10月5日与被�

...

告李四驾驶的上述车辆发生碰撞，

In [62]:
test_message=[{"role": "user", "content": "以法律判决书的风格写同桌把我橡皮弄丢了，要具体真实细节"}]
response = client.chat.completions.create(
    model="gpt-4o-mini", messages=test_message,  max_tokens=1000
)
print(response.choices[0].message.content)

**判决书**

中华人民共和国某市人民法院

民事判决书

（2023）某民初字第123号

案件审理经过：

原告张三（以下简称“原告”），被告李四（以下简称“被告”）。原告因被告在共同学习期间将其个人物品橡皮擦弄丢，特向本院提起诉讼。案件于2023年10月15日公开开庭审理，原告及被告均出庭参与诉讼。

案件事实：

原告在校期间，与被告同为某中学九年级学生，二人于2023年9月1日建立同桌关系。在此期间，原告在课堂上使用的橡皮擦为“XYZ品牌”，颜色为蓝色，大小约为6cm x 3cm，原告在2023年9月10日之前的多次课堂中均有使用该橡皮擦。

2023年9月12日，原告在美术课期间将橡皮擦借给被告使用，借出时被告明确表示会归还。原告于同日下课后发现橡皮擦未归还。原告多次询问被告，但被告答复称未能找到该橡皮擦，并表示会继续寻找。

原告于2023年9月20日再次询问被告，仍未得到满意答复，被告称“可能在教室里掉了”。原告对此表示不满，并要求被告赔偿相应损失。被告则表示不愿意承担赔偿责任，双方因此发生争执。

庭审中，原告提供了两位同班同学的证言，证言内容一致，均确认原告在借给被告橡皮擦后，该橡皮擦未归还。被告则辩称，自己并未故意损坏或丢失原告的橡皮擦，且在借用过程中并未造成损失。

判决意见：

经本院审理查明，原告的橡皮擦在借出后确实未能归还，且被告在明知橡皮擦归属的情况下，未能妥善保管该物品，导致原告的合法权益受到侵害。因此，被告应对此损失承担相应赔偿责任。

依据《中华人民共和国民法典》相关规定，判决如下：

一、被告李四于判决生效之日起三日内，向原告张三赔偿因丢失橡皮擦造成的经济损失人民币30元（按市面同类商品价格计算）。

二、案件受理费由被告负担。

本判决为终审判决，判决自送达之日起生效。

审判长：某某某

审判员：某某某

审判员：某某某

2023年10月20日

（盖章）


In [63]:
test_message=[{"role": "user", "content": "以法律判决书的风格写同桌把我橡皮弄丢了，要具体真实细节"}]
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_message,  max_tokens=1000
)
print(response.choices[0].message.content)

高淳区人民法院

民 事 判 决 书

(2015)高民二初字第149号

原告高淳县实验小学

委托代理人刘翔

委托代理人李勇

被告朱泾镇福东小学

第三人黄志杰

原告高淳县实验小学与被告朱泾镇福东小学学生侵权责任纠纷一案，原告高淳县实验小学、被告朱泾镇福东小学在委托代理人缺席的情况下，经本院主持调解，第三人黄志杰本人自愿与原告高淳县实验小学达成一致协议：黄志杰赔偿原告高淳县实验小学橡皮损失费119元。双方均认可该调解协议，表示自愿履行。请被告朱泾镇福东小学督促第三人黄志杰履行。

如不履行，原告高淳县实验小学可自本调解书送达之日起六个月内，向本院申请强制执行。

被告朱泾镇福东小学须在调解书送达之日起十日内将第三人黄志杰赔偿款支付予原告高淳县实验小学；或第三人黄志杰须在调解书送达之日起十日内将赔偿款支付予原告高淳县实验小学。

否则，原告高淳县实验小学可以依法向本院申请强制执行，被告朱泾镇福东小学及第三人黄志杰也可以自依法应当知道或者应该知道权利被侵害之日起二年内向本院申请保护其合法权益。

审判员 杨蓉

  

二〇一五年十月二十二日

书记员 苏伟


### 如果SFT后感觉在某些领域LLM变傻了?

用餐厅服务员培训来打比方：
1. **过度专注**
- 现象：就像服务员被培训得太死板
- 例子：只会按标准话术接待，遇到特殊情况就懵了

2. **知识遗忘**
- 现象：像新人忘记了基础技能
- 例子：专注记菜单，反而忘了基本的礼貌用语

3. **数据质量问题**
```python
# 不好的训练数据
问：天气怎么样？
答：今天是晴天，温度25度。   # 死板标准答案

# 更好的训练数据
问：天气怎么样？
答：看起来阳光不错，很适合出门。不过可能需要带把伞，下午可能有阵雨。
    # 展现思考和推理能力
```

4. **评估偏差**
- 错误示例：用翻译数据训练，却拿创意写作来测试
- 正确示例：应该用对应任务场景评估

**迭代优化方法**
```
收集数据 → 初步训练 → 实际测试 → 发现问题
↑__________________________|
```


调整这些超参数来改善模型：

1. **批量大小 (batch_size)**
```python
# 场景比喻：老师批改作业
小批量: batch_size = 8
- 像老师每改8份作业就总结一次问题
- 调整更频繁，反应更敏捷
- 适合：数据变化大、细节要求高

大批量: batch_size = 32
- 像老师攒32份作业一起改
- 更稳定，但反应较慢
- 适合：数据模式稳定、追求效率

实践建议：
- 数据复杂/差异大 → 用小批量(8-16)
- 数据规整/相似度高 → 用大批量(32-64)
- GPU内存小 → 被迫用小批量
- 追求训练速度 → 尽量用大批量
```

2. **学习率乘数 (learning_rate_multiplier)**
```python
# 场景比喻：调整学习步伐
保守策略: learning_rate_multiplier = 0.1
- 像放慢脚步仔细观察
- 更稳定，不容易过拟合
- 适合：精调已有能力

激进策略: learning_rate_multiplier = 1.0
- 像大步快走快速探索
- 学习更快，但风险更大
- 适合：全新任务快速适应

实践建议：
if 已有相似任务的基础:
    learning_rate_multiplier = 0.1  # 微微调
elif 全新领域:
    learning_rate_multiplier = 0.5  # 中等步伐
elif 数据量很大:
    learning_rate_multiplier = 1.0  # 放心学习
```

3. **训练轮次 (n_epochs)**
```python
# 场景比喻：复习次数
少轮次: n_epochs = 1-2
- 像快速过一遍知识点
- 防止过拟合
- 适合：数据量大、质量高

多轮次: n_epochs = 3-5
- 像反复钻研加深理解
- 学得更透彻
- 适合：数据量小、需要反复学习

实践建议：
if 大规模高质量数据:
    n_epochs = 1  # 一遍就够
elif 中等规模数据:
    n_epochs = 2  # 复习一遍
elif 小规模数据:
    n_epochs = 3-5  # 多次复习
```


In [93]:
response = client.fine_tuning.jobs.retrieve(job_id)
response

FineTuningJob(id='ftjob-kc7IUjd0IsAFUAzVDnLhnUxU', created_at=1730863716, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:acmr:gov-file:AQR6d7hq', finished_at=1730864085, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-BbylnC34ErFQAfVanaQGLcaB', result_files=['file-VXUJPYaeg4IMgQGmByGpd66D'], seed=774007061, status='succeeded', trained_tokens=503982, training_file='file-lHL2f8P0QT90EljK0xhwr4qJ', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='gov_file')

In [ ]:
import time
class OpenAIFineTuner:
    """
    Class to fine tune OpenAI models
    """
    def __init__(self, training_file_path, model_name, suffix):
        self.training_file_path = training_file_path
        self.model_name = model_name
        self.suffix = suffix
        self.file_object = None
        self.fine_tuning_job = None
        self.model_id = None

    def create_openai_file(self):
        self.file_object = client.files.create(
            file=open(self.training_file_path, "r"),
            purpose="fine-tune",
        )

    def wait_for_file_processing(self, sleep_time=20):
        while self.file_object.status != 'processed':
            time.sleep(sleep_time)
            self.file_object.refresh()
            print("File Status: ", self.file_object.status)

    def create_fine_tuning_job(self):
        self.fine_tuning_job = client.fine_tuning.jobs.create(
            training_file=self.file_object["id"],
            model=self.model_name,
            suffix=self.suffix,
        )

    def wait_for_fine_tuning(self, sleep_time=45):
        while self.fine_tuning_job.status != 'succeeded':
            time.sleep(sleep_time)
            self.fine_tuning_job.refresh()
            print("Job Status: ", self.fine_tuning_job.status)

    def retrieve_fine_tuned_model(self):
        self.model_id = client.fine_tuning.jobs.retrieve(self.fine_tuning_job["id"]).fine_tuned_model
        return self.model_id

    def fine_tune_model(self):
        self.create_openai_file()
        self.wait_for_file_processing()
        self.create_fine_tuning_job()
        self.wait_for_fine_tuning()
        return self.retrieve_fine_tuned_model()

fine_tuner = OpenAIFineTuner(
        training_file_path="your file",
        model_name="gpt-3.5-turbo",
        suffix="name"
    )